In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Reading a PDf File
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('../personal_finance.pdf')
docs=loader.load()
docs

[Document(metadata={'producer': '3.0.24 (5.1.10)', 'creator': 'PyPDF', 'creationdate': '', 'title': 'summarized_content.md', 'moddate': '2025-06-03T05:08:58+02:00', 'source': '../personal_finance.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content="What is Personal Financial Management?\nPersonal financial management encompasses all financial decisions and activities of an\nindividual, including budgeting, insurance, savings, investing, debt servicing, mortgages,\nand more. Managing money successfully is key to financial well-being and includes three\ncritical components:\nManaging income by spending less than you earn, which enables building success\nthrough saving and investing.\nManaging debt by eliminating as much as possible, starting with stopping credit\ncard use and working to pay them off. Financial specialists suggest keeping debt\npayments below twenty percent of income.\nManaging savings and investments, recognizing the distinction between savings\nand inve

Roughly 135 words per chunk

In [3]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [4]:
documents

[Document(metadata={'producer': '3.0.24 (5.1.10)', 'creator': 'PyPDF', 'creationdate': '', 'title': 'summarized_content.md', 'moddate': '2025-06-03T05:08:58+02:00', 'source': '../personal_finance.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content="What is Personal Financial Management?\nPersonal financial management encompasses all financial decisions and activities of an\nindividual, including budgeting, insurance, savings, investing, debt servicing, mortgages,\nand more. Managing money successfully is key to financial well-being and includes three\ncritical components:\nManaging income by spending less than you earn, which enables building success\nthrough saving and investing.\nManaging debt by eliminating as much as possible, starting with stopping credit\ncard use and working to pay them off. Financial specialists suggest keeping debt\npayments below twenty percent of income.\nManaging savings and investments, recognizing the distinction between savings\nand inve

In [ ]:
from langchain_community.llms import Ollama
# import streamlit as st
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


## Prompt Template
full_prompt = """"You are an AI financial asisstant aimed at creating constructive, clear, and sound advice or strategies or tips to help achieve the user's goals. 
You are to take the user's {{goals}}, {{income_level}}, and desired{{time_frame}} to achieve them into consideration. 
For better context, address their {{blockages}}. Offer realistic goal-setting and be sure to suggest a few changes to the plan if deemed beneficial for the user. 
If the user does not provide these info, be sure to ask. For instance, if user provided all info except blockages, ask if they perceive of any road blocks or concerns.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", full_prompt),
        ("user", "Question: {question}")
    ]
)

## streamlit framework
# st.title("Langchain Demo With Gemma Model")
# input_text=st.text_input("What question you have in mind?")


# Use a valid model name you've pulled, e.g., "llama2" or "gemma:2b"
llm = Ollama(model="llama3") 
output_parser = StrOutputParser()
chain = prompt | llm | output_parser

input_text = "I would like to eliminate my debt in 4 months. I have a monthly income of $2000 and my current debt is $5000. I am concerned about my high interest rates."
answer = chain.invoke({"question": input_text})
print(answer)
# if input_text:
#     st.write(chain.invoke({"question":input_text}))

I'd be happy to help you achieve your goal of eliminating your debt within the next 4 months!

First, let's take a closer look at your financial situation:

* Monthly income: $2000
* Current debt: $5000
* Concerns: High interest rates

To tackle this challenge, we'll need to create a solid plan that addresses your concerns and helps you pay off the debt efficiently.

Before we dive into the details, I'd like to ask: What are some of the roadblocks or concerns you have about achieving this goal? Are there any specific challenges you're anticipating, such as difficulty in sticking to a budget or managing expenses?

Once we understand these potential hurdles, we can work together to create a tailored plan that addresses them and helps you stay on track.

Assuming you don't encounter any major roadblocks, here's a possible strategy:

1. **Debt Snowball:** Let's prioritize your debts by focusing on the one with the highest interest rate first. This will help you save money in interest payme

In [7]:
documents[0].page_content

"What is Personal Financial Management?\nPersonal financial management encompasses all financial decisions and activities of an\nindividual, including budgeting, insurance, savings, investing, debt servicing, mortgages,\nand more. Managing money successfully is key to financial well-being and includes three\ncritical components:\nManaging income by spending less than you earn, which enables building success\nthrough saving and investing.\nManaging debt by eliminating as much as possible, starting with stopping credit\ncard use and working to pay them off. Financial specialists suggest keeping debt\npayments below twenty percent of income.\nManaging savings and investments, recognizing the distinction between savings\nand investments, with investments helping to earn money and increase wealth.\nFinancial Goals and Motivation\nEffective financial management begins with careful consideration of your motivations.\nYou must understand why you've decided to organize your finances rather than

In [22]:
documents[0].metadata

{'producer': '3.0.24 (5.1.10)',
 'creator': 'PyPDF',
 'creationdate': '',
 'title': 'summarized_content.md',
 'moddate': '2025-06-03T05:08:58+02:00',
 'source': '../personal_finance.pdf',
 'total_pages': 12,
 'page': 0,
 'page_label': '1'}

In [25]:
list_docs_content = [documents[i].page_content for i in range(len(documents))]
list_docs_content[0]

"What is Personal Financial Management?\nPersonal financial management encompasses all financial decisions and activities of an\nindividual, including budgeting, insurance, savings, investing, debt servicing, mortgages,\nand more. Managing money successfully is key to financial well-being and includes three\ncritical components:\nManaging income by spending less than you earn, which enables building success\nthrough saving and investing.\nManaging debt by eliminating as much as possible, starting with stopping credit\ncard use and working to pay them off. Financial specialists suggest keeping debt\npayments below twenty percent of income.\nManaging savings and investments, recognizing the distinction between savings\nand investments, with investments helping to earn money and increase wealth.\nFinancial Goals and Motivation\nEffective financial management begins with careful consideration of your motivations.\nYou must understand why you've decided to organize your finances rather than

In [ ]:
# !pip install pinecone_datasets

Have to ensure you open Ollama

In [13]:
# Install langchain-community if not already installed

from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="mxbai-embed-large")
text = documents[0].page_content
embedding_sample = embeddings.embed_query(text)
embedding_sample

[0.283488005399704,
 0.09119196236133575,
 -0.46510350704193115,
 -0.0983896553516388,
 -0.2464846521615982,
 0.09814146161079407,
 0.4691699147224426,
 -0.2375659942626953,
 0.8546534180641174,
 0.24043232202529907,
 0.5251124501228333,
 0.18847306072711945,
 -0.018430158495903015,
 0.001955680549144745,
 -0.4786369204521179,
 0.33193743228912354,
 -0.2676042318344116,
 -0.23609520494937897,
 -1.0992200374603271,
 0.06402567774057388,
 -0.17399272322654724,
 0.5271835923194885,
 -0.8983830213546753,
 0.010201171040534973,
 -0.5284500122070312,
 0.13905197381973267,
 -0.20989228785037994,
 0.13980990648269653,
 0.6055675745010376,
 0.9977606534957886,
 -0.25299760699272156,
 -0.18685057759284973,
 0.06422030925750732,
 0.12108878791332245,
 -0.718449056148529,
 -0.12777292728424072,
 0.038249872624874115,
 -0.9436470866203308,
 -0.0400719940662384,
 0.16865947842597961,
 0.6923342943191528,
 -0.38704022765159607,
 0.9443466663360596,
 -0.44379767775535583,
 -0.37543290853500366,
 -0.29

Knowing dimension is critical

In [15]:
len(embedding_sample)

1024

In [35]:
documents[0]

Document(metadata={'producer': '3.0.24 (5.1.10)', 'creator': 'PyPDF', 'creationdate': '', 'title': 'summarized_content.md', 'moddate': '2025-06-03T05:08:58+02:00', 'source': '../personal_finance.pdf', 'total_pages': 12, 'page': 0, 'page_label': '1'}, page_content="What is Personal Financial Management?\nPersonal financial management encompasses all financial decisions and activities of an\nindividual, including budgeting, insurance, savings, investing, debt servicing, mortgages,\nand more. Managing money successfully is key to financial well-being and includes three\ncritical components:\nManaging income by spending less than you earn, which enables building success\nthrough saving and investing.\nManaging debt by eliminating as much as possible, starting with stopping credit\ncard use and working to pay them off. Financial specialists suggest keeping debt\npayments below twenty percent of income.\nManaging savings and investments, recognizing the distinction between savings\nand inves

Refer to:

https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/langchain-retrieval-augmentation.ipynb#scrollTo=zeQnf52sDRfF

In [19]:
from pinecone import Pinecone

# Instantiate a Pinecone client
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [20]:
from pinecone import ServerlessSpec

index_name = 'financial-advisor-index'

if not pc.has_index(name=index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,  # dimensionality of mxbai-embed-large:335m
        metric='dotproduct',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

pc.describe_index(name=index_name)

{
    "name": "financial-advisor-index",
    "metric": "dotproduct",
    "host": "financial-advisor-index-ojlhqno.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1024,
    "deletion_protection": "disabled",
    "tags": null
}

In [21]:
# Instantiate an Index client
index = pc.Index(name=index_name)
index.describe_index_stats()

/opt/miniconda3/envs/myenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'dotproduct',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [40]:
from tqdm import tqdm
from langchain_pinecone import PineconeVectorStore

# Batch and Upsert Documents with tqdm
batch_size = 100 # Adjust batch size based on your needs and Pinecone limits (max 1000 vectors per request)
total_batches = (len(documents) + batch_size - 1) // batch_size # Calculate total number of batches

print(f"Upserting documents in batches of {batch_size}...")

# Initialize PineconeVectorStore (without documents initially)
# This will connect to your existing or newly created index
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

# Use tqdm to show progress
for i in tqdm(range(0, len(documents), batch_size), desc="Upserting to Pinecone"):
    batch = documents[i:i + batch_size]
    try:
        vectorstore.add_documents(batch)
    except Exception as e:
        print(f"\nError during upsert of batch {i//batch_size + 1}: {e}")
        # Implement more robust error handling if needed, e.g., retry logic

print("\nDocuments upserted successfully!")


Upserting documents in batches of 100...


Upserting to Pinecone:   0%|          | 0/1 [00:00<?, ?it/s]

Upserting to Pinecone: 100%|██████████| 1/1 [00:02<00:00,  2.35s/it]


Documents upserted successfully!


In [41]:
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'metric': 'dotproduct',
 'namespaces': {'': {'vector_count': 25}},
 'total_vector_count': 25,
 'vector_type': 'dense'}

In [44]:
from pprint import pprint

query = """
what are some tips to eliminate debt quickly?
"""

documents = vectorstore.similarity_search(
    query=query,
    k=5  # return 5 most relevant docs
)

for doc in documents:
    pprint(doc.__dict__)
    print()

{'id': '4ddee997-5655-43ee-afd8-c55861e40176',
 'metadata': {'creationdate': '',
              'creator': 'PyPDF',
              'moddate': '2025-06-03T05:08:58+02:00',
              'page': 0.0,
              'page_label': '1',
              'producer': '3.0.24 (5.1.10)',
              'source': '../personal_finance.pdf',
              'title': 'summarized_content.md',
              'total_pages': 12.0},
 'page_content': 'What is Personal Financial Management?\n'
                 'Personal financial management encompasses all financial '
                 'decisions and activities of an\n'
                 'individual, including budgeting, insurance, savings, '
                 'investing, debt servicing, mortgages,\n'
                 'and more. Managing money successfully is key to financial '
                 'well-being and includes three\n'
                 'critical components:\n'
                 'Managing income by spending less than you earn, which '
                 'enables 

Runnables:
https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html

Runnable Passthroughs:
https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html

JSON parser:
https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.json.JsonOutputParser.html

Pinecone:
https://python.langchain.com/docs/integrations/vectorstores/pinecone/

Pydantic:
https://docs.pydantic.dev/latest/concepts/fields/

Pinecone Langchain
https://docs.pinecone.io/integrations/langchain



In [ ]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from pydantic import BaseModel, Field # <--- USE THIS LINE INSTEAD
from typing import Optional
from langchain_core.runnables import RunnableLambda
# from langchain.output_parsers import PydanticOutputParser # New import
from langchain_core.output_parsers import PydanticOutputParser


# --- 1. Define the Schema for Extracted Information (remains the same) ---
class FinancialDetails(BaseModel):
    goals: Optional[str] = Field(None, description="The user's primary financial goals (e.g., 'eliminate debt', 'save for retirement').")
    income_level: Optional[str] = Field(None, description="The user's current income level (e.g., '$2000 per month', 'unknown').")
    time_frame: Optional[str] = Field(None, description="The desired time frame to achieve the goals (e.g., '4 months', '5 years').")
    blockages: Optional[str] = Field(None, description="Any perceived roadblocks or concerns preventing the user from achieving their goals (e.g., 'high interest rates', 'lack of savings').")

# --- 2. Create the Pydantic Output Parser ---
parser = PydanticOutputParser(pydantic_object=FinancialDetails)

# --- 3. Define the Extraction Prompt with Format Instructions ---
# We use the parser's get_format_instructions()
extraction_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert at extracting financial details from user queries.\n{format_instructions}"), # Inject instructions here
    ("human", "User query: {query}")
])

# --- 4. Create the Extraction Chain ---
# Instead of `llm.with_structured_output`, we chain the prompt, LLM, and parser.
# We also need to add partial_variables for the format_instructions.
extraction_chain = (
    extraction_prompt.partial(format_instructions=parser.get_format_instructions()) # Inject instructions into the prompt
    | llm # Use the raw LLM, not one configured with .with_structured_output
    | parser # Parse the output of the LLM
)

# --- 5. Define your Main Financial Advice Prompt ---
full_prompt_string = """You are an AI financial assistant aimed at creating constructive, clear, and sound advice or strategies or tips to help achieve the user's goals.
You are to take the user's {goals}, {income_level}, and desired {time_frame} to achieve them into consideration.
Be sure to address their {blockages}. Offer realistic goal-setting and be sure to suggest a few changes to the plan if deemed beneficial for the user.
If the user does not provide these info, be sure to ask. For instance, if user provided all info except blockages, ask if they perceive of any road blocks or concerns.
Refer to given {context} to guide your constructive advice.
"""

financial_advice_prompt = PromptTemplate(
    input_variables=["goals", "income_level", "time_frame", "blockages", "context"],
    template=full_prompt_string
)

# --- 6. Context Formatting Function (remains the same) ---
def format_docs(docs):
    return "\n".join(doc.page_content for doc in docs)

# (Rest of your code for full_prompt_string, financial_advice_prompt, format_docs remains the same)

# --- 7. Build the Complete RAG Chain with Extraction (Slightly adjusted) ---
qa_chain = (
    RunnablePassthrough.assign(
        # First, extract details from the 'question'
        extracted_details=RunnablePassthrough.assign(query=RunnablePassthrough()) | extraction_chain,
        # 'context' comes from the retriever using the original 'question'
        context=RunnablePassthrough.assign(query=RunnablePassthrough()) | vectorstore.as_retriever() | format_docs,
    )
    .assign(
        # Now, merge the extracted details into the main dictionary.
        goals=lambda x: x["extracted_details"].goals or "not provided",
        income_level=lambda x: x["extracted_details"].income_level or "not provided",
        time_frame=lambda x: x["extracted_details"].time_frame or "not provided",
        blockages=lambda x: x["extracted_details"].blockages or "not provided",
    )
    | financial_advice_prompt
    | llm
    | StrOutputParser()
)

# --- Example Usage (remains the same) ---
# result = qa_chain.invoke({
#     "question": "I would like to eliminate my debt in 4 months. I have a monthly income of $2000 and my current debt is $5000. I am concerned about my high interest rates."
# })
# print(result)

In [55]:
qa_chain.invoke({
    "question": "I would like to eliminate my debt in 4 months. I have a monthly income of 8k and my current debt is 5k. I am concerned about my high interest rates.",
    "context": documents
})      

"I'm excited to help you achieve your goals! To get started, I'd like to confirm the details you provided:\n\n* You want to eliminate debt\n* Your monthly income is $8000\n* You want to achieve this in 4 months\n\nPlease confirm if these are correct. Additionally, could you please share any specific concerns or roadblocks you perceive that might impact your ability to achieve your goals?\n\nAssuming these details are accurate, I'd like to offer some initial suggestions:\n\n1. High-interest debt: With high interest rates, it's essential to prioritize paying off debts with the highest interest rates first. This will help minimize the amount of interest paid over time.\n2. Budgeting: To achieve your goal of eliminating debt in 4 months, you'll need to create a realistic budget that allocates a significant portion of your income towards debt repayment. We can work together to determine a suitable allocation percentage.\n3. Income generation: Considering your high monthly income, we might b

In [57]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

qa_with_sources.invoke({
    "question": "I would like to eliminate my debt in 4 months. I have a monthly income of 8k and my current debt is 5k. I am concerned about my high interest rates.",
    "context": documents
})      

{'question': 'I would like to eliminate my debt in 4 months. I have a monthly income of 8k and my current debt is 5k. I am concerned about my high interest rates.',
 'context': [Document(id='4ddee997-5655-43ee-afd8-c55861e40176', metadata={'creationdate': '', 'creator': 'PyPDF', 'moddate': '2025-06-03T05:08:58+02:00', 'page': 0.0, 'page_label': '1', 'producer': '3.0.24 (5.1.10)', 'source': '../personal_finance.pdf', 'title': 'summarized_content.md', 'total_pages': 12.0}, page_content="What is Personal Financial Management?\nPersonal financial management encompasses all financial decisions and activities of an\nindividual, including budgeting, insurance, savings, investing, debt servicing, mortgages,\nand more. Managing money successfully is key to financial well-being and includes three\ncritical components:\nManaging income by spending less than you earn, which enables building success\nthrough saving and investing.\nManaging debt by eliminating as much as possible, starting with stop

Prompt History

In [58]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# Define a simple in-memory store for chat message history
store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(llm, get_session_history)
config={"configurable": {"session_id": "chat1"}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content="I want to save reduce my expenses by 20% in the next 6 months. My current monthly expenses are 6k. I am concerned about my high bills for grocery and dining out."),],
    config=config,
)

response

Error in RootListenersTracer.on_llm_end callback: KeyError('message')


"Let's break down your goal to save 20% of your current monthly expenses, which is $6,000.\n\nTo calculate your target savings amount, you can multiply your current monthly expenses by the percentage you want to reduce:\n\n$6,000 (monthly expenses) x 0.20 (20%) = $1,200 per month\n\nThis means you're aiming to save approximately $1,200 each month for the next six months to reach your goal.\n\nNow, let's focus on those high bills for grocery and dining out. Here are some suggestions to help you reduce these expenses:\n\nGrocery Expenses:\n\n1. Plan your meals: Take time to plan out your meals for the week. This will help you avoid buying unnecessary items and reduce food waste.\n2. Create a grocery list: Make a list of the ingredients you need for your planned meals. Stick to your list when you're at the store to avoid impulse purchases.\n3. Shop sales: Check the weekly ads for your local grocery stores and plan your shopping trip around the items that are on sale.\n4. Buy in bulk: Purc

In [62]:
groq_api_key=os.getenv("GROQ_API_KEY")

groq_api_key

from langchain_groq import ChatGroq

model=ChatGroq(model="llama3-8b-8192", groq_api_key=groq_api_key)

from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Krish and I am a Chief AI Engineer")])

AIMessage(content='Nice to meet you, Krish! As a Chief AI Engineer, you must be working on some exciting projects that are pushing the boundaries of artificial intelligence. What areas of AI are you particularly interested in or working on? Are you focused on machine learning, natural language processing, computer vision, or something else?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 23, 'total_tokens': 85, 'completion_time': 0.051666667, 'prompt_time': 0.003329405, 'queue_time': 0.020190875, 'total_time': 0.054996072}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run--371a6923-cfc0-4f63-af3e-a7259fbc36f4-0', usage_metadata={'input_tokens': 23, 'output_tokens': 62, 'total_tokens': 85})

https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/langchain-retrieval-agent.ipynb#scrollTo=SRWda2C4JZIZ